## <b>Tensorflow basics</b>

#### Tensors

Tensorflow operrates on multidiamentional array or tensors represented as tf.Tensor objects.
Here two diamentional tensor :

In [1]:
import tensorflow as tf

x= tf.constant([[1,2,3],
               [4,5,6]])
print(x)
print(x.shape)
print(x.dtype)


tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)
(2, 3)
<dtype: 'int32'>


#### Operations

In [2]:
x + x

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 2,  4,  6],
       [ 8, 10, 12]])>

In [3]:
5 * x

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 5, 10, 15],
       [20, 25, 30]])>

In [4]:
x @ tf.transpose(x)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[14, 32],
       [32, 77]])>

In [5]:
tf.concat([x, x, x], axis=0)

<tf.Tensor: shape=(6, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6],
       [1, 2, 3],
       [4, 5, 6],
       [1, 2, 3],
       [4, 5, 6]])>